<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from importnb import Notebook
with Notebook(): 
    from che4 import Props, qtox, xtoq

import numpy as np
from scipy.optimize import root, brentq

### NRTL Activity Coefficient Model

$ln (\gamma_i) = \dfrac{ \sum\limits_{j=1}^{n}x_j \tau_{ji}G_{ji}}{\sum\limits_{k=1}^{n}x_kG_{ki}} + 
\sum\limits_{j=1}^{n} \dfrac{x_jG_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}} \left( \tau_{ij} - 
\dfrac{\sum\limits_{m=1}^{n} x_m \tau_{mj} G_{mj}}{\sum\limits_{k=1}^{n} x_kG_{kj}} \right)$

$\tau_{ji} = A_{ji} + \dfrac{B_{ji}}{T} + C_{ji}ln(T)+D_{ji}T$

$G_{ji}=exp \left( - \alpha_{ji} \tau_{ji} \right)$

$\alpha_{ij}=\alpha_{ji}$

In [12]:
def NRTL_gamma_loop(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    lng = np.zeros_like(x)
    xtauGdivxG = np.zeros_like(x)
    xG = np.zeros_like(x)
    term2 = np.zeros_like(x)
    N = x.size
    for i in range(N):
        xtauG = 0.
        for j in range(N):
            xtauG += x[j]*tau[j,i]*G[j,i]
            xG[i] += x[j]*G[j,i]
        xtauGdivxG[i] = xtauG/xG[i]
    for i in range(N):
        for j in range(N):
            term2[i]+= x[j]*G[i,j]/xG[j]*(tau[i,j] - xtauGdivxG[j])

    return np.exp(xtauGdivxG + term2)

def NRTL_gamma_einsum(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = np.einsum('j,ji->i',x,tau*G)
    xG = np.einsum('j,ji',x,G)
    xtauGdivxG=xtauG/xG
    term2=np.einsum('j,ij->i',x,G*(tau-xtauGdivxG[None,:])/xG[None,:] )
    return np.exp(xtauGdivxG + term2)

def NRTL_gamma_matvec(x,T):
    x=x.reshape((1,-1))
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = x@ (tau*G)
    xG = x@G
    xtauGdivxG=xtauG/xG
    a=G*(tau-xtauGdivxG[None,:])/xG[None,:]
    lng= xtauGdivxG + a@(x.squeeze())
    return np.exp(lng.flatten())

In [13]:
p = Props(['Methanol','Ethanol','Water'])

In [27]:
z=np.array([1/3, 1/3, 1/3])

In [45]:
def bubblePy_NRTL(x, T):
    Pi=x*NRTL_gamma_matvec(x,T)*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P 

def bubbleT_NRTL(x, P):
    Tb=p.Tb(P)
    Tlo, Thi = np.min(Tb), np.max(Tb)

    def eq(T):
        return bubblePy_NRTL(x,T)[0]-P

    return brentq(eq, Tlo-40, Thi+40)

def dewP_ideal(y, T):
    pvap = p.Pvap(T)
    P =  1/(np.sum(y/pvap))
    x = y * P/pvap
    return P, x

def dewT_ideal(y, P):
    Tb=p.Tb(P)
    Tlo, Thi = np.min(Tb), np.max(Tb)

    def eq(T):
        return dewP_ideal(y,T)[0]-P

    return brentq(eq, Tlo-40, Thi+40)


def dewP_NRTL(y, T):

    def eq(v):
        x = qtox(v[:-1])
        P = v[-1]
        return x*NRTL_gamma_matvec(x,T)*p.Pvap(T)-y*P

    P_ideal, x_ideal = dewP_ideal(y,T)
    v_guess=np.append(xtoq(x_ideal),P_ideal)
    sol=root(eq,v_guess).x
    return qtox(sol[:-1]), sol[-1]



In [46]:
dewP_NRTL(z,400)

(DeviceArray([0.2084266 , 0.22785431, 0.56371909], dtype=float64),
 514139.98501294805)

In [39]:
dewP_ideal(z,500.)

(DeviceArray(4068696.56011698, dtype=float64),
 DeviceArray([0.20631222, 0.27936238, 0.51432541], dtype=float64))

In [40]:
dewT_ideal(z,4068696)

499.9999923165434

In [35]:
xtoq(np.array([0.1,0.3,0.6]))

DeviceArray([-1.79175947, -0.69314718], dtype=float64)

In [37]:
qtox(np.array([-1.79175947, -0.69314718]))

DeviceArray([0.1, 0.3, 0.6], dtype=float64)